In [3]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [4]:
RANDOM_STATE = 42

In [5]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

X.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [7]:
# подсказка https://nagornyy.me/it/regressionnye-modeli-v-python/
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X_train, y_train)
y_test_pred = slr.predict(X_test)
from sklearn.metrics import r2_score
print('R^2 test: {:.7f}'.format(r2_score(y_test, y_test_pred)))

R^2 test: 0.6687595


In [8]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
y_test_pred = ridge.predict(X_test)
R2_ridge = r2_score(y_test, y_test_pred)
print('R^2 test: {:.7f}'.format(R2_ridge))

R^2 test: 0.6686244


In [9]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_test_pred = lasso.predict(X_test)
R2_lasso = r2_score(y_test, y_test_pred)
print('Lasso R^2 test: {:.7f}'.format(R2_lasso))

Lasso R^2 test: 0.6569713


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
from sklearn.linear_model import RidgeCV
clf = RidgeCV(alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]).fit(X_test, y_test)
R2_ridgeCV = clf.score(X_test, y_test)
print('RidgeCV R^2 test: {:.7f}'.format(R2_ridgeCV))

from sklearn.linear_model import LassoCV
clf = LassoCV(alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]).fit(X_test, y_test)
R2_lassoCV = clf.score(X_test, y_test)
print('LassoCV R^2 test: {:.7f}'.format(R2_lassoCV))


RidgeCV R^2 test: 0.7655114
LassoCV R^2 test: 0.7196445


In [11]:
# R2 вырос
print('R2_lassoCV-R2_lasso = ',(R2_lassoCV-R2_lasso))
print('R2_ridgeCV-R2_ridge = ',(R2_ridgeCV-R2_ridge))

R2_lassoCV-R2_lasso =  0.06267320152991263
R2_ridgeCV-R2_ridge =  0.09688698353788339


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
pipeline = Pipeline([('scaler',StandardScaler()), ('clf', Ridge(alpha=10**-5))])
pipeline.fit(X_train, y_train)
r2_ridge_scaler = pipeline.score(X_test, y_test)
print('Pipeline r2_ridge_scaler',r2_ridge_scaler)
print('r2_ridge_scaler-R2_ridgeCV', r2_ridge_scaler-R2_ridgeCV)
pipeline = Pipeline([('scaler',StandardScaler()), ('clf', Lasso(alpha=10**-5))])
pipeline.fit(X_train, y_train)
r2_lasso_scaler = pipeline.score(X_test, y_test)
print ('Pipeline r2_lasso_scaler',r2_lasso_scaler)
print ('r2_lasso_scaler - R2_lassoCV',r2_lasso_scaler - R2_lassoCV)


Pipeline r2_ridge_scaler 0.6687594905710065
r2_ridge_scaler-R2_ridgeCV -0.09675190516901855
Pipeline r2_lasso_scaler 0.668759038334717
r2_lasso_scaler - R2_lassoCV -0.05088544341758916


In [13]:
pipeline = Pipeline([('scaler',MinMaxScaler()), ('clf', Ridge(alpha=10**-5))])
pipeline.fit(X_train, y_train)
r2_ridge_scaler = pipeline.score(X_test, y_test)
r2_ridge_scaler
print('MinMaxScaler r2_ridge_scaler',r2_ridge_scaler)
print('r2_ridge_scaler-R2_ridgeCV', r2_ridge_scaler-R2_ridgeCV)
pipeline = Pipeline([('scaler', MinMaxScaler()), ('clf', Lasso(alpha=10**-5))])
pipeline.fit(X_train, y_train)
r2_lasso_scaler = pipeline.score(X_test, y_test)
r2_lasso_scaler
print ('MinMaxScaler r2_lasso_scaler',r2_lasso_scaler)
print ('r2_lasso_scaler - R2_lassoCV',r2_lasso_scaler - R2_lassoCV)


MinMaxScaler r2_ridge_scaler 0.6687596289762543
r2_ridge_scaler-R2_ridgeCV -0.09675176676377073
MinMaxScaler r2_lasso_scaler 0.6687605073677362
r2_lasso_scaler - R2_lassoCV -0.05088397438456993


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [14]:
from sklearn.model_selection import GridSearchCV
scalers = [('MinMax', MinMaxScaler()),
           ('Standard', StandardScaler())]
models = [('Ridge', Ridge()),
          ('Lasso', Lasso())]
alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]
CV=10
for scaler_name, scaler in scalers:
    for model_name, model in models:
        pipe = Pipeline([('scaler', scaler), ('model', model)])
        grid_search = GridSearchCV(pipe, param_grid=[{'model__alpha': alphas}], scoring='r2', cv=CV)
        grid_search.fit(X_train, y_train)
        score = grid_search.score(X_test, y_test)
        print(
            f"{scaler_name} + {model_name}: alpha_opt = {grid_search.best_params_['model__alpha']}, r2 score = {score}")

MinMax + Ridge: alpha_opt = 1, r2 score = 0.6764100365423611
MinMax + Lasso: alpha_opt = 0.01, r2 score = 0.6676993404117642
Standard + Ridge: alpha_opt = 10.0, r2 score = 0.6659677905050349
Standard + Lasso: alpha_opt = 0.01, r2 score = 0.6681815922762605


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [15]:
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

scalers = [('MinMax', MinMaxScaler()),
           ('Standard', StandardScaler())]
models = [('Ridge', Ridge()),
          ('Lasso', Lasso())]
polyfeatures = [('feature mults without feature squares', PolynomialFeatures(degree=2, interaction_only=True)),
                ('feature mults + feature squares', PolynomialFeatures(degree=2))]

for scaler_name, scaler in scalers:
    for model_name, model in models:
        for polyf_name, polyfeature in polyfeatures:
            pipe = Pipeline([('scaler', scaler), ('poly', polyfeature), ('model', model)])
            grid_search = GridSearchCV(pipe, param_grid=[{'model__alpha': alphas}], scoring='r2', cv=CV)
            grid_search.fit(X_train, y_train)
            score = grid_search.score(X_test, y_test)
            print(
                f"{scaler_name} + {model_name} + {polyf_name}: alpha_opt = {grid_search.best_params_['model__alpha']}, r2 score = {score}")
# результат улучшился

MinMax + Ridge + feature mults without feature squares: alpha_opt = 0.01, r2 score = 0.8572252837849037
MinMax + Ridge + feature mults + feature squares: alpha_opt = 0.01, r2 score = 0.8335358555037755
MinMax + Lasso + feature mults without feature squares: alpha_opt = 0.0001, r2 score = 0.8401697806745037
MinMax + Lasso + feature mults + feature squares: alpha_opt = 0.001, r2 score = 0.8390581680518304
Standard + Ridge + feature mults without feature squares: alpha_opt = 10.0, r2 score = 0.8496468217328315
Standard + Ridge + feature mults + feature squares: alpha_opt = 10.0, r2 score = 0.8180465877243659
Standard + Lasso + feature mults without feature squares: alpha_opt = 0.01, r2 score = 0.8509794967776149
Standard + Lasso + feature mults + feature squares: alpha_opt = 0.01, r2 score = 0.8138518691835616


7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [16]:
param_grid = dict(scaler=[None, MinMaxScaler(), StandardScaler()],
                  poly=[None, PolynomialFeatures(degree=2, interaction_only=True), PolynomialFeatures(degree=2)],
                  model=[Ridge(), Lasso()],
                  model__alpha=alphas)

pipe = Pipeline([('scaler', 'passthrough'), ('poly', 'passthrough'), ('model', 'passthrough')])
grid_search = GridSearchCV(pipe, param_grid, scoring='r2', cv=CV)
grid_search.fit(X_train, y_train)
score = grid_search.score(X_test, y_test)
print(f"opt_scaler: {grid_search.best_params_['scaler']}\n"
      f"opt_poly:{grid_search.best_params_['poly']}\n"
      f"opt_model:{grid_search.best_params_['model']}\n"
      f"opt_score:{score}"
      )

opt_scaler: StandardScaler()
opt_poly:PolynomialFeatures(interaction_only=True)
opt_model:Ridge(alpha=10.0)
opt_score:0.8496468217328315


http://archive.ics.uci.edu/ml/datasets/Adult

In [17]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [18]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [19]:
df = data.copy()
N = len(df.columns)
X = df.iloc[:, :N - 1]
y = df.iloc[:, -1].apply(lambda x: 1.0 if x == '>50K' else 0.0)

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [20]:
df.shape == df.dropna().shape

True

10. Выберите колонки с числовыми и категориальными переменными.

In [21]:
from pandas.api.types import is_string_dtype

max_n_unique = 0.000005 * df.shape[0]
cat, non_cat = [], []
for col in X.columns:
    if df[col].nunique() < max_n_unique or is_string_dtype(df[col]):
        cat.append(col)
    else:
        non_cat.append(col)

print(f'категориальные:{cat}\nчисловые:{non_cat}')

категориальные:[1, 3, 5, 6, 7, 8, 9, 13]
числовые:[0, 2, 4, 10, 11, 12]


11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

t = [('num', MinMaxScaler(), non_cat), ('cat', OneHotEncoder(), cat)]
transformer = ColumnTransformer(transformers=t)
X_train1_sparse = transformer.fit_transform(X.copy())
X1 = pd.DataFrame(X_train1_sparse.toarray())
X1.head()

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,0.301370,0.044131,0.800000,0.02174,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=RANDOM_STATE)
most_freq_val = y.value_counts().idxmax()
X1_most_freq_train = X1.loc[y == most_freq_val]
y1_most_freq = y.loc[y == most_freq_val]

log_reg = LogisticRegression()
log_reg.fit(X1_train, y1_train)
y1_pred = log_reg.predict(X1_most_freq_train)
accuracy = accuracy_score(y1_most_freq, y1_pred)
f1_score_w = f1_score(y1_most_freq, y1_pred, average='weighted')
f1_score_b = f1_score(y1_most_freq, y1_pred, average='binary')
f1_score_m = f1_score(y1_most_freq, y1_pred, average='micro')
print(
    f'accuracy: {accuracy}\nf1 score(weighted):{f1_score_w}\nf1 score(binary):{f1_score_b}\nf1 score(micro):{f1_score_m}')

accuracy: 0.9335486475575293
f1 score(weighted):0.9656324383012485
f1 score(binary):0.0
f1 score(micro):0.9335486475575293


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC

models = [LogisticRegression(), SVC(), LinearSVC()]
metrics = ['accuracy', 'f1']

for model in models:
    for metric in metrics:
        score = cross_val_score(model, X1, y, cv=CV, scoring=metric).mean()
        print(f'model: {model}, metric: {metric}, score: {score}')

model: LogisticRegression(), metric: accuracy, score: 0.8512142211067493
model: LogisticRegression(), metric: f1, score: 0.656585452057147
model: SVC(), metric: accuracy, score: 0.8410180549023947
model: SVC(), metric: f1, score: 0.6225568402859734
model: LinearSVC(), metric: accuracy, score: 0.852667876306566
model: LinearSVC(), metric: f1, score: 0.6571268426576358


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [25]:
from sklearn.impute import SimpleImputer

inputer = SimpleImputer(missing_values='?', strategy='most_frequent')
inputer = inputer.fit_transform(X.copy())
X_cleared = pd.DataFrame(inputer)
X_cleared.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [26]:
X_cleared_sparse = transformer.fit_transform(X_cleared.copy())
X2 = pd.DataFrame(X_cleared_sparse.toarray())

models = [LogisticRegression(), LinearSVC()]
metrics = ['accuracy', 'f1']

for model in models:
    for metric in metrics:
        score = cross_val_score(model, X2, y, cv=CV, scoring=metric).mean()
        print(f'model: {model}, metric: {metric}, score: {score}')
# результат улучшить не удалось

model: LogisticRegression(), metric: accuracy, score: 0.8506408870021971
model: LogisticRegression(), metric: f1, score: 0.6538607252852946
model: LinearSVC(), metric: accuracy, score: 0.8511322874935976
model: LinearSVC(), metric: f1, score: 0.6517974174872275


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [27]:
X_dropped = df.copy().replace('?', np.nan).dropna()
X1_dropped = X_dropped.iloc[:, :N - 1]
y = X_dropped.iloc[:, -1].apply(lambda x: 1.0 if x == '>50K' else 0.0)

X1_dropped_sparse = transformer.fit_transform(X1_dropped)
X2_dropped = pd.DataFrame(X1_dropped_sparse.toarray())

for model in models:
    for metric in metrics:
        score = cross_val_score(model, X2_dropped, y, cv=CV, scoring=metric).mean()
        print(f'model: {model}, metric: {metric}, score: {score}')
# результат немного ухудшился

model: LogisticRegression(), metric: accuracy, score: 0.8471099407099377
model: LogisticRegression(), metric: f1, score: 0.6604944060167736
model: LinearSVC(), metric: accuracy, score: 0.8483703520157381
model: LinearSVC(), metric: f1, score: 0.6613169761301343


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [28]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

models = [RandomForestClassifier(), GradientBoostingClassifier()]
for model in models:
    for metric in metrics:
        score = cross_val_score(model, X2_dropped, y, cv=CV, scoring=metric).mean()
        print(f'model: {model}, metric: {metric}, score: {score}')
# результат улучшился

model: RandomForestClassifier(), metric: accuracy, score: 0.8493654184622054
model: RandomForestClassifier(), metric: f1, score: 0.6735577973376395
model: GradientBoostingClassifier(), metric: accuracy, score: 0.8630314878898486
model: GradientBoostingClassifier(), metric: f1, score: 0.6880056062412362


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.